Below you will find the routine to compute the ensemble mean position. It is in FORTRAN but I think you can understand the algorithm (let me know if you still have troubles to translate). Basically the inputs are knpf (dimensions of the vector of rlatpf and rlonpf), rlatpf & rlonpf the vectors containing the latitudes and longitudes of the ensemble for one time  step ( this routine is called for each forecast time step!). The output are the pair rlatmean and rlonmean (average position for that time step). Let me know if you have any other questions!

knps -> dimensions of the vector of rlatpf and rlonpf \
rlatpf, rlonpf -> vectors containing the latitudes and longitudes of the ensemble for one time step \
rlatmean, rlonmean -> output average position for that time step

In [59]:
import pdbufr
import math
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [60]:
def meanposit(knpf, rlatpf, rlonpf):
    rpi = math.acos(0.0)
    rnomin = 0.0
    rdenom = 0.0
    rphisum = 0.0
    
    for k in range(knpf):
        rlat = rlatpf[k] * rpi / 180.0
        rlon = rlonpf[k] * rpi / 180.0
        rcosphi = math.cos(rlat)
        rnomin += rcosphi * rlon
        rdenom += rcosphi
        rphisum += rlat
    
    rlabda = rnomin / rdenom
    rlonmean = rlabda * 180.0 / rpi
    
    rnomin = 0.0
    repsilon = 0.0
    
    for k in range(knpf):
        rlat = rlatpf[k] * rpi / 180.0
        rlon = rlonpf[k] * rpi / 180.0
        rcosphi = math.cos(rlat)
        rnomin += rcosphi * (rlabda - rlon) ** 2
    
    repsilon = rnomin / (2.0 * knpf)
    rphimean = rphisum / float(knpf)
    rphi = rphimean + repsilon * math.sin(rphimean)
    rlatmean = rphi * 180.0 / rpi
    
    return rlatmean, rlonmean

In [62]:
def create_storms_df():
    # Load cyclone dataframe with Mean sea level pressure value
    df_storms = pdbufr.read_bufr('track_data/tc_test_track_data.bufr',
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "pressureReducedToMeanSeaLevel"))
    # Load cyclone dataframe with Wind speed at 10m value
    df1 = pdbufr.read_bufr('track_data/tc_test_track_data.bufr',
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "windSpeedAt10M"))
    # Add the Wind speed at 10m column to the storms dataframe 
    df_storms["windSpeedAt10M"] = df1.windSpeedAt10M
    # Storms with number higher than 10 are not real storms (according to what Fernando said)
    drop_condition = df_storms.stormIdentifier < '11'
    df_storms = df_storms[drop_condition]
    return df_storms

df_storms = create_storms_df()
df_storms.head()

,stormIdentifier,ensembleMemberNumber,latitude,longitude,pressureReducedToMeanSeaLevel,windSpeedAt10M
0,07E,1,16.6,-128.9,99900.0,18.5
1,07E,1,16.3,-130.1,100200.0,18.0
2,07E,1,16.4,-131.3,100200.0,17.0
3,07E,1,16.3,-132.4,100500.0,16.0
4,07E,1,16.4,-133.2,100300.0,14.4


In [65]:
# Function that returns the list of coordinates for the mean forecast track
def mean_forecast_track(df_storm):
    
    # Create 2 dataframe with latitude and logitude coordinates for each ensemble as columns
    members = df_storm.ensembleMemberNumber.unique()
    df_lat_tracks = pd.DataFrame()
    df_lon_tracks = pd.DataFrame()
    for member in members:
        df_track = df_storm[df_storm.ensembleMemberNumber == member]
        df_track.reset_index(inplace=True)
        df_lat_tracks[f'latitude{member}'] = df_track.latitude
        df_lon_tracks[f'longitude{member}'] = df_track.longitude
    
    # Cycle through the rows of df_lat_track and df_lon_tracks to compute the average track lat,lon
    mean_track_coord = []
    for t in range(len(df_lat_tracks)):
        lat = df_lat_tracks.iloc[t].dropna().to_numpy()
        lon = df_lon_tracks.iloc[t].dropna().to_numpy()
        mean_lat_lon = meanposit(len(lat), lat, lon)
        mean_track_coord.append(mean_lat_lon)
        
    return mean_track_coord

df_storm = df_storms[df_storms.stormIdentifier == '09E']
mean_forecast_track(df_storm)

[(13.778862793638012, -105.24039801421178),
 (14.284700117288995, -106.09040320697372),
 (14.790465797031512, -107.20001175658544),
 (15.253922915272895, -108.4672708568927),
 (15.81933826055671, -109.46534509877222),
 (16.404032804202934, -110.38267754487515),
 (17.0675532692854, -111.22891098128844),
 (17.834894049607122, -112.03085336249225),
 (18.71960150928682, -112.79243598714463),
 (19.506168992879537, -113.44256577010347),
 (20.358190525390302, -113.94258128373616),
 (21.33329842335623, -114.3158572592538),
 (22.52568972687106, -114.60456555881302),
 (23.741215156006824, -114.79141318984392),
 (25.204877692820034, -115.07631880642934),
 (26.739614525860066, -115.51511161449696),
 (28.39940316872781, -116.04372256055159),
 (30.23458307851418, -116.73695256693138),
 (31.73315387926842, -117.5121565671089),
 (32.21804648331987, -118.20180888082443),
 (31.90201607647228, -119.56959364001948),
 (32.064959829146176, -120.32476971974668),
 (31.56754584329959, -121.43386990477526),
 (3